# Erstellen der Kollektionen in Q-drant mit einem Late-Interaction Modell

Hier werden die Daten für die Vektordatenbank vorbereitet und in diese geladen. Als Modell wird hier ein Late Interaction Modell verwendet.

- Modell: jinaai/jina-colbert-v2
- Link: https://huggingface.co/jinaai/jina-colbert-v2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
RAW_DATA_PATH = "drive/MyDrive/Uni/Master/Masterthesis/Data/raw_data/"
PROD_DATA_PATH = "drive/MyDrive/Uni/Master/Masterthesis/Data/topics/"
DATA_PATH = "drive/MyDrive/Uni/Master/Masterthesis/Data/"

## Laden der Expertenprofile

In [ ]:
import json

# Pfad zur Datei
file_path = DATA_PATH + "enhanced_expert_profiles.json"

# JSON direkt einlesen
with open(file_path, "r", encoding="utf-8") as f:
    expert_data = json.load(f)

# Ausgabe prüfen
print(type(expert_data))  # sollte list oder dict sein
print(len(expert_data))   # Anzahl der Einträge prüfen
print(expert_data[0])     # Ersten Eintrag anzeigen
print(expert_data[0].keys())     # Ersten Eintrag anzeigen

<class 'list'>
307
{'branches': ['Technologie', 'Medien & Werbung', 'Unternehmensdienstleistungen'], 'companyLocationCity': 'Lennestadt', 'companyLocationStreet': 'Hagener Straße 64', 'companyLocationZip': 57368.0, 'companyName': 'lenne.Tech GmbH', 'companyTypes': ['Dienstleistung'], 'companyWebsite': 'https://lenne.tech', 'description': 'Seit über 15 Jahren begleite ich digitale Projekte von der Konzeption bis zur "schlüsselfertigen" Anwendung. Als Mitglied im Team von lenne.Tech (https://lenne.tech) entwickle ich Apps & Webanwendungen und unterstütze Unternehmen beim Aufbau sowie bei der Weiterbildung von Entwicklungsteams. Im Rahmen unserer lenne.Learning Akademie (https://lennelearning.de) organisiere ich gemeinsam mit anderen Tutoren & Mentoren das Recruiting sowie das Onboarding neuer Entwickler für Partnerunternehmen.', 'employeeOfInstitutionNames': [], 'firstName': 'Ege', 'gender': 'MALE', 'id': '65acfb5a3897d6f0e6506db1', 'jobTitle': 'Softwareentwickler', 'lastName': 'Siebert'

## Aufsetzen der Kollektionen

Quelle: https://qdrant.tech/course/essentials/day-5/colbert-multivectors/

Cluster: https://cloud.qdrant.io/accounts/35801706-940d-46e2-903a-fc1d0f88cda5/clusters/004ad81f-9442-4601-8579-089a7f905391/overview

In [ ]:
!pip install -U sentence-transformers
!pip install -U qdrant-client
!pip install fastembed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


### Vorbereiten Expertenprofile

In [ ]:
# only keep original data fields from experts
keys_to_remove = ["full_text_fields_used", "full_text_fields_used_fixed", "full_text_word_count", "full_text_fields_used_sorted", "validation_issues"]

expert_data = [
    {k: v for k, v in obj.items() if k not in keys_to_remove}
    for obj in expert_data
]

expert_data[0].keys()

dict_keys(['branches', 'companyLocationCity', 'companyLocationStreet', 'companyLocationZip', 'companyName', 'companyTypes', 'companyWebsite', 'description', 'employeeOfInstitutionNames', 'firstName', 'gender', 'id', 'jobTitle', 'lastName', 'projectsDescription', 'skills', 'title', 'full_text'])

### Laden von Qdrant Client

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(
    url="url_here",
    api_key="api_key",
)


In [ ]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='expert_data_fulltext_sparse'), CollectionDescription(name='expert_data_structured_dense'), CollectionDescription(name='expert_data_structured_and_fulltext_dense'), CollectionDescription(name='expert_data_fulltext_dense'), CollectionDescription(name='expert_data_structured_and_fulltext_sparse'), CollectionDescription(name='expert_data_fulltext_late'), CollectionDescription(name='expert_data_late_test'), CollectionDescription(name='expert_data_structured_sparse'), CollectionDescription(name='expert_data_structured_late')])

#### Erstellen der Kollektionen


In [ ]:
from fastembed import LateInteractionTextEmbedding

late_model = LateInteractionTextEmbedding("jinaai/jina-colbert-v2")
print(late_model.embedding_size)  # sollte 128 sein

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

onnx/model.onnx_data:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/3.98M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

128


In [ ]:
from qdrant_client.models import VectorParams, Distance

if not client.collection_exists("expert_data_structured_late"):
  client.create_collection(
      collection_name='expert_data_structured_late',
      vectors_config={
          "late_interaction": models.VectorParams(
              size=late_model.embedding_size,              # 128
              distance=models.Distance.COSINE,
              multivector_config=models.MultiVectorConfig(
                  comparator=models.MultiVectorComparator.MAX_SIM
              ),
              # HNSW für Multivector deaktivieren, da es eher fürs Reranking genutzt wird
              hnsw_config=models.HnswConfigDiff(m=0),
          )
      },
  )

if not client.collection_exists("expert_data_fulltext_late"):
  client.create_collection(
      collection_name='expert_data_fulltext_late',
      vectors_config={
          "late_interaction": models.VectorParams(
              size=late_model.embedding_size,              # 128
              distance=models.Distance.COSINE,
              multivector_config=models.MultiVectorConfig(
                  comparator=models.MultiVectorComparator.MAX_SIM
              ),
              # HNSW für Multivector deaktivieren, da es eher fürs Reranking genutzt wird
              hnsw_config=models.HnswConfigDiff(m=0),
          )
      },
  )

if not client.collection_exists("expert_data_structured_and_fulltext_late"):
  client.create_collection(
      collection_name='expert_data_structured_and_fulltext_late',
      vectors_config={
          "late_interaction": models.VectorParams(
              size=late_model.embedding_size,              # 128
              distance=models.Distance.COSINE,
              multivector_config=models.MultiVectorConfig(
                  comparator=models.MultiVectorComparator.MAX_SIM
              ),
              # HNSW für Multivector deaktivieren, da es eher fürs Reranking genutzt wird
              hnsw_config=models.HnswConfigDiff(m=0),
          )
      },
  )

### Laden der Expertendaten

In [ ]:
import copy

expert_data_copy = copy.deepcopy(expert_data)

# remove full_text
keys_to_remove = ["full_text"]

expert_data_without_full_text = [
    {k: v for k, v in obj.items() if k not in keys_to_remove}
    for obj in expert_data_copy
]

print(expert_data_without_full_text[0].keys())

dict_keys(['branches', 'companyLocationCity', 'companyLocationStreet', 'companyLocationZip', 'companyName', 'companyTypes', 'companyWebsite', 'description', 'employeeOfInstitutionNames', 'firstName', 'gender', 'id', 'jobTitle', 'lastName', 'projectsDescription', 'skills', 'title'])


In [ ]:
points = []

for idx, doc in enumerate(expert_data_without_full_text):
    text = json.dumps(doc, ensure_ascii=False)
    multivector = next(late_model.embed(text))

    print(idx)

    points.append(
        models.PointStruct(
            id=idx,
            vector={
                "late_interaction": multivector,
            },
            payload=doc,
        )
    )

client.upload_points(
    collection_name="expert_data_structured_late",
    points=points,
    batch_size=32,
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
points_fulltext = []

for idx, doc in enumerate(expert_data):
    full_text = doc.get("full_text", "")
    if not full_text:
        full_text = ""  # zur Sicherheit

    multivector = next(late_model.embed(full_text))
    print(idx)

    points_fulltext.append(
        models.PointStruct(
            id=idx,
            vector={
                "late_interaction": multivector,
            },
            payload=doc,
        )
    )

client.upload_points(
    collection_name="expert_data_fulltext_late",
    points=points_fulltext,
    batch_size=32,
)


Ege Siebert ist Softwareentwickler mit umfangreichen Kompetenzen in Softwareentwicklung, Web-Entwicklung, IT-Systemadministration, Programmierkenntnissen, Design, Cloud-Computing, DevOps, Content Management, Data Analysis und Systemverwaltung. Seit über 15 Jahren begleitet er digitale Projekte von der Konzeption bis zur schlüsselfertigen Anwendung. Als Mitglied des Teams von lenne.Tech (https://lenne.tech) entwickelt er Apps und Webanwendungen und unterstützt Unternehmen beim Aufbau sowie bei der Weiterbildung von Entwicklungsteams. Im Rahmen der lenne.Learning Akademie (https://lennelearning.de) organisiert er gemeinsam mit anderen Tutoren & Mentoren das Recruiting sowie das Onboarding neuer Entwickler für Partnerunternehmen. Die von lenne.Tech betreuten Projekte richten sich an Unternehmen, Selbstständige, Bildungsinstitutionen sowie Gründungs- und Technikinteressierte und behandeln Fragen rund um App- und Webentwicklung sowie Digitalisierung. Der Arbeitskontext umfasst Technologie, 

KeyboardInterrupt: 

In [ ]:
points_structured_fulltext = []

for idx, doc in enumerate(expert_data):
    print(doc.get("full_text", ""))
    text = json.dumps(doc, ensure_ascii=False)
    multivector = next(late_model.embed(text))

    print(idx)

    points_structured_fulltext.append(
        models.PointStruct(
            id=idx,
            vector={
                "late_interaction": multivector,
            },
            payload=doc,
        )
    )

client.upload_points(
    collection_name="expert_data_structured_and_fulltext_late",
    points=points_structured_fulltext,
    batch_size=32,
)



Ege Siebert ist Softwareentwickler mit umfangreichen Kompetenzen in Softwareentwicklung, Web-Entwicklung, IT-Systemadministration, Programmierkenntnissen, Design, Cloud-Computing, DevOps, Content Management, Data Analysis und Systemverwaltung. Seit über 15 Jahren begleitet er digitale Projekte von der Konzeption bis zur schlüsselfertigen Anwendung. Als Mitglied des Teams von lenne.Tech (https://lenne.tech) entwickelt er Apps und Webanwendungen und unterstützt Unternehmen beim Aufbau sowie bei der Weiterbildung von Entwicklungsteams. Im Rahmen der lenne.Learning Akademie (https://lennelearning.de) organisiert er gemeinsam mit anderen Tutoren & Mentoren das Recruiting sowie das Onboarding neuer Entwickler für Partnerunternehmen. Die von lenne.Tech betreuten Projekte richten sich an Unternehmen, Selbstständige, Bildungsinstitutionen sowie Gründungs- und Technikinteressierte und behandeln Fragen rund um App- und Webentwicklung sowie Digitalisierung. Der Arbeitskontext umfasst Technologie, 